# Дипломный проект «Data Scientist. Аналитика. Средний уровень»

Анализ результатов опроса клиентов компании "МегаФон" в целях выявления зависимости между оценкой удовлетворенности качеством связи и техническими показателями. 

In [162]:
import pandas as pd
import numpy as np
import scipy.stats as st

import plotly.graph_objects as go
import plotly.express as px
import ast

In [163]:
df = pd.read_csv('megafon.csv')
df.head()

,user_id,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming Download Throughput(Kbps),Video Streaming xKB Start Delay(ms),Web Page Download Throughput(Kbps),Web Average TCP RTT(ms)
0,1,5,NaN,775.48846,360.13,86.56,3.93,1859.15,2309,1007.82,83
1,2,5,4,861.96324,3023.54,411.18,1.27,667.47,2080,255.36,425
2,3,1,4,261.11860,790.96,34.20,1.79,1079.60,6367,535.85,485
3,4,8,3,179.18564,2590.97,325.88,0.80,7053.81,3218,1221.02,51
4,5,2,"2, 3, 4",351.99208,731.61,223.54,1.15,4550.38,1767,2336.56,68


# Часть 1

Прежде всего необходимо проанализировать ответы на первый вопрос "Насколько в целом Вы удовлетворены качеством связи у МегаФона за последний месяц?" 

In [164]:
# список ответов клиентов на первый вопрос
df['Q1'].unique().tolist() 

['5',
 '1',
 '8',
 '2',
 '3',
 '9',
 '10',
 '7',
 '4',
 '11',
 '6',
 '2, 9',
 '0',
 '1, 3',
 '19',
 '15',
 nan,
 '1, 6',
 '***** ** ***',
 '3 - дер.Ширяево Волоколамского района, 9 - в Москве',
 '10, 9',
 'Чем даль ше,тем лучше.Спасибо за ваш труд.Оценка 10 !',
 'ОЦЕНКА-3/НЕВАЖНО/',
 'Отвратительно',
 'Я ценой услуг не удовлетворен',
 'Пока не понял',
 '3, 9',
 '5, 6',
 '0, 1, 5',
 '5, 7',
 'Hi',
 '4. Тульская область Заокский район. Романовские дачи связи почти нет',
 'Немагу дать атценку денги незашто снимаеть скоро выклучаю',
 '10, 50',
 'Очень  хорошо. Обслуживания  я довольно. Спасибо вам.555',
 '?',
 'Поохое',
 'Когда в Москве-10 а когда в калужской области в деревне Бели-1',
 'Нет',
 'Да',
 'Ужасно',
 '3 тройка, связь отвратительная, жалко платить за такой тарив',
 'Чдтчдтччдтччч',
 '3, 7',
 '20, 89031081392',
 '1, 8',
 'Без з',
 '10, 5',
 '2, 5',
 'Я в Смол. Области живу сейчас, не пользуюсь телефоном совсем']

Столбец содержит также нерелевантные данные, например: текстовые ответы, несколько значений и не входящие в диапазон числа.

Для корректного анализа необходимо провести очистку данных.

In [165]:
# Удаление нерелевантных данных

marks = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
df = df.loc[df['Q1'].isin(marks)]
df['Q1'] = df['Q1'].astype(int)
df['Q1'].unique().tolist()

[5, 1, 8, 2, 3, 9, 10, 7, 4, 6]

In [166]:
# Визуализация распределения оценок
df_marks = df.groupby('Q1').agg({'user_id' : 'count'})

fig = go.Figure(data=[go.Bar(x = df_marks.index, y = df_marks['user_id'], marker_color = 'royalblue', text=df_marks['user_id'], textposition='outside')])          
fig.update_layout(title = 'Распределение оценок')
fig.update_xaxes({'dtick' : 1}, title = 'Оценки')
fig.update_yaxes(title = 'Количество') 
fig.show()

Исходя из графика, можно сделать вывод, что преобладает оценка "10", но также наблюдается большое количество неудовлетворительных оценок (1-8). 
Для компании важно минимизировать количество низких оценок, поэтому целесообразно оценить, что может оказывать влияние на удовлетворенность клиентов.

In [167]:
df_correlation = df.corr()
df_correlation['Q1'].sort_values(ascending = False)
df_correlation

,user_id,Q1,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming Download Throughput(Kbps),Video Streaming xKB Start Delay(ms),Web Page Download Throughput(Kbps),Web Average TCP RTT(ms)
user_id,1.000000,0.003759,-0.002434,0.005629,-0.003396,-0.033186,-0.009411,-0.019838,-0.005669,0.034608
Q1,0.003759,1.000000,-0.006246,0.088240,0.067721,-0.093386,0.108803,-0.100187,0.083663,-0.117002
Total Traffic(MB),-0.002434,-0.006246,1.000000,0.142469,0.018312,-0.092062,0.038646,-0.049636,0.028740,-0.074521
Downlink Throughput(Kbps),0.005629,0.088240,0.142469,1.000000,0.156120,-0.192326,0.576065,-0.225456,0.266168,-0.189598
Uplink Throughput(Kbps),-0.003396,0.067721,0.018312,0.156120,1.000000,-0.099840,0.170372,-0.080932,0.121661,-0.142160
Downlink TCP Retransmission Rate(%),-0.033186,-0.093386,-0.092062,-0.192326,-0.099840,1.000000,-0.241937,0.193828,-0.193402,0.133720
Video Streaming Download Throughput(Kbps),-0.009411,0.108803,0.038646,0.576065,0.170372,-0.241937,1.000000,-0.367639,0.386492,-0.256265
Video Streaming xKB Start Delay(ms),-0.019838,-0.100187,-0.049636,-0.225456,-0.080932,0.193828,-0.367639,1.000000,-0.174785,0.232962
Web Page Download Throughput(Kbps),-0.005669,0.083663,0.028740,0.266168,0.121661,-0.193402,0.386492,-0.174785,1.000000,-0.225465
Web Average TCP RTT(ms),0.034608,-0.117002,-0.074521,-0.189598,-0.142160,0.133720,-0.256265,0.232962,-0.225465,1.000000


Проанализировав представленные технические показатели, можно предположить, что пинг при просмотре web-страниц (Web Average TCP RTT(ms)) оказывает значимое влияние на общую оценку деятельности, т.к. задержка при загрузке web-страниц может привести к недовольству клиента и, как следствие, к получению плохой оценки компании. 

**Задача 1.** Проверить, существует ли зависимость между оценками клиентов и показателем "Web Average TCP RTT(ms)".

*Разведочный анализ*

In [168]:
df_corr = df.groupby('Q1').agg({'Web Average TCP RTT(ms)' : ['min', 'max', 'mean']})
df_corr

Web Average TCP RTT(ms)                  
                       min   max        mean
Q1                                          
1                       29  3471  258.894737
2                       35  1535  198.648810
3                       22  2178  225.886154
4                       35  1737  180.105691
5                        0  2128  190.269231
6                        0  2394  192.742574
7                       40  2178  164.445000
8                       36  2533  169.910653
9                       30  2295  173.689076
10                       0  2982  165.600473

In [169]:
fig = go.Figure(data=go.Scatter(x=df['Q1'], y=df['Web Average TCP RTT(ms)'], mode='markers'))
fig.update_layout(title = 'Корреляция ответов пользователей и Web Average TCP RTT(ms)')
fig.update_xaxes({'dtick' : 1}, title = 'Оценки')
fig.update_yaxes(title = 'Пинг при просмотре web-страниц') 
fig.show()

Как следует из таблицы и полученного графика, высокий показатель пинга при просмотре web-страниц наблюдается среди пользователей, которые в дальнейшем поставили самые низкие оценки. Это может быть сигналом наличия взаимосвязи между показателями, для проверки которой необходимо применить статистические методы.

*Статистический тест*

Нулевая и альтернативная гипотезы:
H0 - показатели независимы;
H1 - существует зависимость между показателями

Для проверки гипотезы об отсутсвии взаимосвязи подходит коэффициент корреляции Пирсона.

In [170]:
pearson_coef, p_value = st.pearsonr(df['Q1'], df['Web Average TCP RTT(ms)'])
print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value) 

if p_value < 0.05: 
   print(" we are rejecting null hypothesis")
else:
  print("we are accepting null hypothesis")

Pearson Correlation Coefficient:  -0.11700181996382522 and a P-value of: 8.597298515431464e-11
 we are rejecting null hypothesis


Коэффициент корреляции Пирсона равный -0.12 свидетельствует о слабой отрицательной связи между оценкой удовлетворенности клиента и пингом при загрузке web-страниц (Web Average TCP RTT(ms)).

P-value меньше 0 и следовательно меньше 0,05. Можно сделать вывод о том, что при 5% уровне значимости нулевая гипотеза отвергается, зависимость есть.
Таким образом, предположение о зависимости оценки пользователей от Web Average TCP RTT(ms) подтвердилось. Выявлена слабая отрицательная связь - при увеличении времени задержки в загрузке web-страниц растет недовольство пользователей и, как следствие, снижаются оценки.

# Часть 2

Далее целесообразно рассмотреть соотношение довольных и недовольных клиентов. 

In [171]:
df_marks['mark'] = df_marks.index
df_marks['mark'] = df_marks['mark'].astype(int)
df_marks['good'] = df_marks['mark'] >= 9
df_marks['good'] = df_marks['good'].replace({True: 'Да', False: 'Нет'})
df_marks = df_marks.groupby('good').agg({'user_id' : 'count'})

In [182]:
fig = go.Figure(data=[go.Bar(x = df_marks.index, y = df_marks['user_id'], marker_color = 'royalblue', width = 0.5)])
                
fig.update_layout(title = 'Распределение ответов')
fig.update_xaxes({'dtick' : 1}, title = 'Полностью удовлетворены качеством связи')
fig.update_yaxes(title = 'Количество ответов') 
fig.show()

На графике видно, что количество недовольных клиентов значительно превышает количество довольных клиентов. Для того, чтобы разобраться с самыми распространенными жалобами клиентов, целесообразно проанализировать ответы на второй вопрос "Что именно Вам не позволило поставить оценку выше?" 

**Задача 2** Найти среднее значение среди ответов (жалоб) клиентов.

*Разведочный анализ*

In [173]:
df = df.dropna()
df = df[df['Q2'] != '0, 05, 2, 27, 7']
df['Q2'] = df.Q2.apply(lambda x: ast.literal_eval(str(x)))
df = df.explode('Q2')
df['Q2'] = df['Q2'].astype(int)
df

,user_id,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming Download Throughput(Kbps),Video Streaming xKB Start Delay(ms),Web Page Download Throughput(Kbps),Web Average TCP RTT(ms)
1,2,5,4,861.96324,3023.54,411.18,1.27,667.47,2080,255.36,425
2,3,1,4,261.11860,790.96,34.20,1.79,1079.60,6367,535.85,485
3,4,8,3,179.18564,2590.97,325.88,0.80,7053.81,3218,1221.02,51
4,5,2,2,351.99208,731.61,223.54,1.15,4550.38,1767,2336.56,68
4,5,2,3,351.99208,731.61,223.54,1.15,4550.38,1767,2336.56,68
...,...,...,...,...,...,...,...,...,...,...,...
3108,3109,3,3,171.52629,670.32,40.94,2.35,1711.54,2780,954.91,251
3108,3109,3,4,171.52629,670.32,40.94,2.35,1711.54,2780,954.91,251
3108,3109,3,5,171.52629,670.32,40.94,2.35,1711.54,2780,954.91,251
3110,3111,6,1,827.74515,1841.90,373.53,1.21,5675.93,1905,2361.88,202


In [174]:
# Как и в предыдущей задаче остаются только релевантные ответы (предлагалось выбрать варианты ответов, представленные в виде чисел от 1 до 7)
answers = ['1', '2', '3', '4', '5', '6', '7']
df = df.loc[df['Q2'].isin(answers)]
df

,user_id,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming Download Throughput(Kbps),Video Streaming xKB Start Delay(ms),Web Page Download Throughput(Kbps),Web Average TCP RTT(ms)
1,2,5,4,861.96324,3023.54,411.18,1.27,667.47,2080,255.36,425
2,3,1,4,261.11860,790.96,34.20,1.79,1079.60,6367,535.85,485
3,4,8,3,179.18564,2590.97,325.88,0.80,7053.81,3218,1221.02,51
4,5,2,2,351.99208,731.61,223.54,1.15,4550.38,1767,2336.56,68
4,5,2,3,351.99208,731.61,223.54,1.15,4550.38,1767,2336.56,68
...,...,...,...,...,...,...,...,...,...,...,...
3108,3109,3,3,171.52629,670.32,40.94,2.35,1711.54,2780,954.91,251
3108,3109,3,4,171.52629,670.32,40.94,2.35,1711.54,2780,954.91,251
3108,3109,3,5,171.52629,670.32,40.94,2.35,1711.54,2780,954.91,251
3110,3111,6,1,827.74515,1841.90,373.53,1.21,5675.93,1905,2361.88,202


In [175]:
df_complains = df.groupby('Q2').agg({'user_id' : 'count'})

fig = go.Figure(data=[go.Bar(x = df_complains.index, y = df_complains['user_id'], marker_color = 'royalblue', text=df_complains['user_id'], textposition='outside')])
                
fig.update_layout(title = 'Распределение жалоб')
fig.update_xaxes({'dtick' : 1}, title = 'Жалобы')
fig.update_yaxes(title = 'Количество') 
fig.show()

In [177]:
# Среднее значение ответов на второй вопрос
df['Q2'].mean()

2.9874949576442114

Чаще всего клиенты высказывали недовольство по пунктам 3 (плохое качество связи в зданиях, торговых центрах и т.д.) и 1 (недозвоны, обрывы при звонках). Предположим, что пункт 3 - средний ответ при вопросе о жалобах у всех клиентов МегаФон.


*Статистический тест*

Нулевая и альтернативная гипотезы:
H0 - средний ответ на вопрос о жалобах равен 3;
H1 - средний ответ на вопрос о жалобах не равен 3

Для проверки гипотезы можно использовать t-test с одной выборкой.

In [16]:
marks = df['Q2'].tolist()

marks_mean = np.mean(marks)
print(marks_mean)
tset, pval = st.ttest_1samp(marks, 3)

print('p-value',pval)

if pval < 0.05: 
   print(" we are rejecting null hypothesis")
else:
  print("we are accepting null hypothesis")

2.9874949576442114
p-value 0.6829429431164097
we are accepting null hypothesis


P-value больше α (при уровне значимости 5%), следовательно H0 не отвергается. 

Доверительный интервал:

In [17]:
st.t.interval(0.95, len(marks)-1, loc=np.mean(marks), scale=st.sem(marks))

(2.927466823057917, 3.047523092230506)

На основании полученных данных можно сделать вывод, что самая вероятная жалоба клиентов - это плохое качество связи в зданиях, торговых центрах и т.п. (ответ № 3).
Доверительный интервал для этой величины от 2.93 до 3.04.
С учетом изложенного, можно порекомендовать компании обратить внимание на эту проблему.

# Часть 3

Для компании важно, что пользователи не только были довольны предоставляемыми услугами, но и как можно активнее ими пользовались. Например, увеличение трафика передачи данных демонстрирует насколько активно абонент использует интернет.
Проверим, от чего зависит увеличение этого показателя.

**Задача 3** Найти взаимосвязь между объемом трафика передачи данных и другими известными показателями.

*Разведочный анализ*

In [152]:
df_correlation['Total Traffic(MB)'].sort_values(ascending=False)

Total Traffic(MB)                            1.000000
Downlink Throughput(Kbps)                    0.142469
Video Streaming Download Throughput(Kbps)    0.038646
Web Page Download Throughput(Kbps)           0.028740
Uplink Throughput(Kbps)                      0.018312
user_id                                     -0.002434
Q1                                          -0.006246
Video Streaming xKB Start Delay(ms)         -0.049636
Web Average TCP RTT(ms)                     -0.074521
Downlink TCP Retransmission Rate(%)         -0.092062
Name: Total Traffic(MB), dtype: float64

Показатель средней скорости "к абоненту" (Downlink Throughput) предположительно оказывает наибольшее влияние на объем трафика (Total Traffic) - коэффициент корреляции 0.17. Представим эту взаимосвязь на графике.


In [134]:
fig = px.scatter(df, x='Total Traffic(MB)', y='Downlink Throughput(Kbps)', trendline='ols', trendline_color_override='red', title='Взаимосвязь объема трафика и средней скорости "к абоненту"')
fig.show()

Линия тренда показывает, что при росте средней скорости "к абоненту" увеличивается объем трафика передачи данных.

*Статистический анализ*
Нулевая и альтернативная гипотезы:
H0 - показатели независимы;
H1 - существует зависимость между показателями

В представленных данных наблюдаются выбросы, поэтому проверки гипотезы целесообразно использовать коэффициент корреляции Спирмена.

In [53]:
data1, data2 = df['Total Traffic(MB)'], df['Downlink Throughput(Kbps)']
corr, pval = st.spearmanr(data1, data2)

print(corr, 'p-value',pval)

if pval < 0.05: 
   print(" we are rejecting null hypothesis")
else:
  print("we are accepting null hypothesis")

0.25484310211642053 p-value 4.764183977657443e-38
 we are rejecting null hypothesis


Статистический тест сообщает о слабой положительной корреляции со значением 0,25. Значение p близко к нулю, что означает, что при 95% достоверности можно отвергнуть нулевую гипотезу о том, что показатели независимы. Предположение о взаимосвязи показателей "Total Traffic(MB)" и "Downlink Throughput(Kbps)", выдвинутое по итогам разведочного анализа, подтвердилось.

# Выводы

С учетом проведенного анализа можно сделать следующие выводы:
* Значительная часть абонентов МегаФон не удовлетворена качеством связи с рассматриваемый период. Можно сказать, что большое влияние на оценки клиентов оказывает показатель Web Average TCP RTT(ms). При увеличении времени задержки в загрузке web-страниц растет недовольство пользователей и, как следствие, снижаются оценки;
* При более подробном рассмотрении жалоб пользователей (причин, которые не позволили поставить оценку выше) выявлено, что средняя оценка равна 3, что соответсвтует проблеме "плохое качество связи в зданиях, торговых центрах и т.п.";
* Наблюдается положительная корреляция объема передачи данных (Total Traffic(MB)) и средней скорости к абоненту (Downlink Throughput(Kbps)). Таким образом, при увеличении средней скорости к абоненту предположительно пользователь будет более активно использовать мобильный интернет, что положительно скажется на прибыли компании.